In [192]:
################################################################################
##### NO NEED TO RUN
##### This cell used to prepare and export 'data' and 'model'
################################################################################


# pip install sentence-transformers sklearn dash numpy gunicorn

## EXPORT movie_lists AND document_embeddings
# import pandas as pd
# import numpy as np

# tmp = pd.read_csv('./movie_vectors/output.csv')
# import pickle
# import json

# with open('./movie_vectors/movie_lists', 'wb') as fp:
#   movie_lists = tmp.title.values
#   pickle.dump(movie_lists, fp)

# with open('./movie_vectors/document_embeddings', 'wb') as fp:
#   document_embeddings = [json.loads(x) for x in tmp.document_embeddings.values]
#   pickle.dump(document_embeddings, fp)

# with open('./movie_vectors/pairwise_similarities', 'wb') as fp:
#   pairwise_similarities = cosine_similarity(document_embeddings)
#   pickle.dump(pairwise_similarities, fp)

# with open('./others/stopwords', 'wb') as fp:
#   stopwords_english = stopwords.words('english')
#   pickle.dump(stopwords_english, fp)

# with open('model.pickle', 'wb') as pkl:
#   sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
#   pickle.dump(sbert_model, pkl)


# import requests
# import pandas as pd

# df = pd.read_csv('./tmdb_5000_movies.csv')
# poster_urls = []
# for movie_id in df.id.values:
#   url = "https://api.themoviedb.org/3/movie/{}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US".format(movie_id)
#   data = requests.get(url)
#   data = data.json()
#   try:
#     poster_path = data['poster_path']
#     full_path = "https://image.tmdb.org/t/p/w500" + poster_path
#   except:
#     full_path = ""
#   poster_urls.append(full_path)

# with open('./data/poster_urls_lists', 'wb') as fp:
#   pickle.dump(np.array(poster_urls), fp)

In [11]:
!pip install jupyter-dash sentence-transformers

     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 4.0 MB 20.6 MB/s 
     |████████████████████████████████| 1.2 MB 45.7 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 880 kB 48.3 MB/s 
     |████████████████████████████████| 6.6 MB 44.4 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=1dc1378e1ffc41a190aab1b0eeb01db638596c61896b3e8b32b0cb19819de6b4
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=80c3c99cafab29c6ef7a353c9a2a8dde81daf1e83ddadb7a37405534c1e901ac
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [2]:
##############################################
# LIBRARIES
##############################################

from sklearn.metrics.pairwise import cosine_similarity
import re
import pickle

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import numpy as np

In [3]:
##############################################
# FUNCTIONS
##############################################

def prepare_text(x):
  return " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l)


In [12]:
##############################################
# DATA
##############################################

with open('./data/stopwords', 'rb') as fp:
  stop_words_l = pickle.load(fp)

with open('./data/movie_lists', 'rb') as fp:
  movie_lists = pickle.load(fp)

with open('./data/poster_urls_lists', 'rb') as fp:
  poster_lists = pickle.load(fp)

with open('./data/document_embeddings', 'rb') as fp:
  document_embeddings = pickle.load(fp)

pairwise_similarities = cosine_similarity(document_embeddings)

# The model.pickle file wasn't uploaded to Github due to file size.
# But those who wants to run this the demo can use the code below.
# from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

# This is for those who has model.pickle file.
with open('./model/model.pickle', 'rb') as fp:
  sbert_model = pickle.load(fp)

In [13]:
app = JupyterDash(__name__)


##############################################
# APP LAYOUT
##############################################

app.layout = html.Div([
          html.H1('Movie Recommendation'),
          
          html.Div(children=[dcc.Dropdown(id="movie-lists",
                                          options=movie_lists,
                                          multi=False)]
                   ),
          html.Div(children=[html.Div(children=[html.H3(id='focused-movie',
                                                        style={'margin': 'auto'}),
                                                html.Div(id='focused-poster')],
                                      style={'textAlign': 'center', 'flexBasis': '100%'}),
                             html.Div(id='similar-movies',
                                      style={'textAlign': 'center', 'flexBasis': '100%', 'display': 'flex', 'width': '100%', 'justifyContent': 'space-around'})
                   ],
                   style={'display': 'flex', 'width': '90%', 'justifyContent': 'space-around'}
                   ),
          

          html.H2("We will recommend movies that are close to the story you describe."),
          html.Div(dcc.Input(id='overview', type='text',
                             placeholder='Describe the movie you want to see... (keyword, overview, story)',
                             value='Ice magic princess who lives with sister',
                             style={'width': '80%'})
          ),
          html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
          html.Div(id="movie",
                   children=[html.Div(children=[html.H3(id='moviename1', style={'marginBottom': 0}),
                                                html.H4(id='moviescore1', style={'margin': 'auto'}),
                                                html.Div(id='poster1')],
                                      style={'textAlign': 'center', 'flexBasis': '100%'}),
                             html.Div(children=[html.H3(id='moviename2', style={'marginBottom': 0}),
                                                html.H4(id='moviescore2', style={'margin': 'auto'}),
                                                html.Div(id='poster2')],
                                      style={'textAlign': 'center', 'flexBasis': '100%'}),
                             html.Div(children=[html.H3(id='moviename3', style={'marginBottom': 0}),
                                                html.H4(id='moviescore3', style={'margin': 'auto'}),
                                                html.Div(id='poster3')],
                                      style={'textAlign': 'center', 'flexBasis': '100%'})],
                   style={'display': 'flex', 'width': '90%', 'justifyContent': 'space-around'}
                   )
          ])


##############################################
# APP CALLBACKS
##############################################

@app.callback([Output('moviename1', 'children'),
               Output('moviename2', 'children'),
               Output('moviename3', 'children'),
               Output('moviescore1', 'children'),
               Output('moviescore2', 'children'),
               Output('moviescore3', 'children'),
               Output('poster1', 'children'),
               Output('poster2', 'children'),
               Output('poster3', 'children')],
              Input('submit-button-state', 'n_clicks'),
              State('overview', 'value'),
              prevent_initial_call=True)
def recommendation(_, text):
  topn = 3
  prepared_text = prepare_text(text)
  embedded_text = [list(sbert_model.encode(prepared_text))]

  similarity = cosine_similarity(embedded_text, document_embeddings)[0]
  similar_idxs = np.argsort(similarity)[::-1]

  similarity_scores = list(similarity[similar_idxs[:topn]])
  recommended_movies = list(movie_lists[similar_idxs[:topn]])
  recommended_posters = list(poster_lists[similar_idxs[:topn]])
              
  scores = [' (' + str(round(similarity_scores[i]*100, 2)) + '%)' for i in range(topn)]

  poster_divs = [html.Img(style={'maxHeight': '400px'}, src=url) for url in recommended_posters]

  return recommended_movies + scores + poster_divs


@app.callback([Output('focused-movie', 'children'),
               Output('focused-poster', 'children'),
               Output('similar-movies', 'children')],
              Input('movie-lists', 'value'))
def similarity(focus_movie):
  topn = 3

  focus_movie_idx = list(movie_lists).index(focus_movie)
  focus_poster_url = poster_lists[focus_movie_idx]
  focus_poster = html.Img(style={'maxHeight': '400px'}, src=focus_poster_url)

  pairwise = pairwise_similarities[focus_movie_idx]  
  similar_idxs = np.argsort(pairwise)[::-1]
  similar_scores = list(pairwise[similar_idxs[1:topn+1]])
  similar_movies = list(movie_lists[similar_idxs[1:topn+1]])
  similar_posters = list(poster_lists[similar_idxs[1:topn+1]])

  similar_div_list = [html.Div(children=[html.H3(similar_movies[i],
                                                 style={'marginBottom': 0}),
                                         html.H4('(' + str(round(similar_scores[i]*100, 2)) + '%)',
                                                 style={'margin': 'auto'}),
                                         html.Div(html.Img(style={'maxHeight': '250px'},
                                                      src=similar_posters[i]))
                                         ],
                               style={'textAlign': 'center', 'flexBasis': '100%', 'margin': '10px'}) for i in range(topn)]

  similar_div = html.Div(children=[html.H5("Similar movies"),
                                   html.Div(children=[similar_div_list[0],
                                                      similar_div_list[1],
                                                      similar_div_list[2]],
                                            style={'display': 'flex', 'width': '90%', 'justifyContent': 'space-around'}),
                                   ]
                         )

  return focus_movie, focus_poster, similar_div


app.run_server('inline', debug=True)


<IPython.core.display.Javascript object>